# Inference Model

In [16]:
# Imports
import os
import sys
import subprocess
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time
from datetime import datetime

# Model Imports
import category_encoders as ce
import xgboost as xgb
import sklearn as sk

In [ ]:
# Dummy Data

dt_test = datetime(2024, 8, 15, 10, 30)

Information = {
    "time_info": dt_test,
    "PULocation": 100,
    "DOLocation": 100,
    "trip_distance": 1000.0,    
}

In [ ]:
# Input function from API



In [ ]:
# Base Preprocess
def encode_info(data):
    
    date = data["time_info"]
    PULocationID = data["PULocation"]
    DOLocationID = data["DOLocation"]
    trip_distance = data["trip_distance"]
    
    # Get day number, month number
    day = date.day
    month = date.month
    hour = date.hour
    
    # Get day of the week
    day_of_week = date.weekday()
    
    # Get weekend or weekday
    weekend = 0
    if day_of_week == 5 or day_of_week == 6:
        weekend = 1
        
    data_encode = {
        "day": day,
        "month": month,
        "hour": hour,
        "day_of_week": day_of_week,
        "weekend": weekend,
        "PULocationID": PULocationID,
        "DOLocationID": DOLocationID,
        "trip_distance": trip_distance
    }
    
    return data_encode
    
    
# Circular Encoding
def encode_circular(data):
    
    day_of_week = data["day_of_week"]
    hour = data["hour"]
    
    # Circular encoding for day of week
    dow_sin = np.sin((day_of_week / 7) * 2 * np.pi)
    dow_cos = np.cos((day_of_week / 7) * 2 * np.pi)
    
    # Circular encoding for hour of the day
    hour_sin = np.sin((hour / 24) * 2 * np.pi)
    hour_cos = np.cos((hour / 24) * 2 * np.pi)
    
    # Create a new dictionary with the circular encoded values
    data_encode = {
        "dow_sin": dow_sin,
        "dow_cos": dow_cos,
        "hour_sin": hour_sin,
        "hour_cos": hour_cos,
        "day_of_month": data["day"],
        "month": data["month"],
        "weekend": data["weekend"],
        "trip_distance": data["trip_distance"],
        "PULocationID": data["PULocationID"],
        "DOLocationID": data["DOLocationID"]
    }
    
    return data_encode

def run_model_xgb(data, path):
    model = xgb.XGBRegressor()
    model.load_model(path)
    
    # Encode the input data
    data_encoded = encode_info(data)
    data_encoded = encode_circular(data_encoded)
    feature_names = model.get_booster().feature_names
    data_df = pd.DataFrame([data_encoded], columns=feature_names)
    data_df = data_df.reindex(columns=feature_names, fill_value=0)
    
    # Make the prediction
    prediction = model.predict(data_df)
    
    return prediction[0]

def run_model_xgb_24(data, path):
    data_encoded = encode_info(data)
    
    model = xgb.XGBRegressor()
    model.load_model(path)
    feature_names = model.get_booster().feature_names
    
    predictions = []
    
    # Run the model for 24 hours of the day  
    for hour in range(24):
        data_temp = data_encoded.copy()
        data_temp["hour"] = hour
        data_temp = encode_circular(data_temp)
        data_df = pd.DataFrame([data_temp], columns=feature_names)
        data_df = data_df.reindex(columns=feature_names, fill_value=0)
        
        # Make the prediction
        prediction = model.predict(data_df)
        predictions.append(prediction[0])
        
    return predictions
    

In [33]:
# Run for total amount 
preds = run_model_xgb_24(Information, 'models/total_amount_base_model.json')

# Print the predictions and format them to 0.2f
print("Predictions for total amount:")
for i in range(24):
    print(f"Hour {i}: {preds[i]:.2f}")

# Run for travel time
preds = run_model_xgb_24(Information, 'models/travel_time_base_model.json')

# Print the predictions in a readable format min, sec
print("Predictions for travel time:")
for i in range(24):
    minutes = int((preds[i] % 3600) // 60)
    seconds = int(preds[i] % 60)
    print(f"Hour {i}: {minutes}m {seconds}s")

Predictions for total amount:
Hour 0: 47.50
Hour 1: 47.17
Hour 2: 46.93
Hour 3: 46.52
Hour 4: 45.92
Hour 5: 46.02
Hour 6: 45.05
Hour 7: 45.11
Hour 8: 45.67
Hour 9: 46.07
Hour 10: 46.91
Hour 11: 47.11
Hour 12: 47.05
Hour 13: 46.60
Hour 14: 46.42
Hour 15: 42.95
Hour 16: 44.07
Hour 17: 44.28
Hour 18: 44.05
Hour 19: 48.25
Hour 20: 48.38
Hour 21: 48.84
Hour 22: 48.65
Hour 23: 48.20
Predictions for travel time:
Hour 0: 24m 51s
Hour 1: 24m 5s
Hour 2: 24m 51s
Hour 3: 24m 50s
Hour 4: 24m 44s
Hour 5: 25m 8s
Hour 6: 26m 21s
Hour 7: 26m 31s
Hour 8: 27m 25s
Hour 9: 28m 20s
Hour 10: 28m 22s
Hour 11: 28m 44s
Hour 12: 29m 28s
Hour 13: 29m 47s
Hour 14: 30m 39s
Hour 15: 30m 38s
Hour 16: 30m 10s
Hour 17: 30m 11s
Hour 18: 29m 0s
Hour 19: 27m 40s
Hour 20: 27m 17s
Hour 21: 26m 58s
Hour 22: 26m 30s
Hour 23: 25m 53s
